# Lab SQL 9
In this lab we will find the customers who were active in consecutive months of May and June. Follow the steps to complete the analysis.


In [1]:
# import packages

import pymysql
from sqlalchemy import create_engine
import pandas as pd
import getpass
import numpy as np
password = getpass.getpass()

········


In [2]:
# connecting sakila

connection_string = "mysql+pymysql://root:" + password + "@localhost/sakila"
engine = create_engine(connection_string)

## Create a table `rentals_may` to store the data from rental table with information for the month of May.

In [3]:
rentals_may = ()

## Insert values in the table `rentals_may` using the table rental, filtering values only for the month of May.
 

In [4]:
# select all rentals from may that have either been rented or returned in may

rentals_may = pd.read_sql_query("""
    SELECT *
    FROM rental
    WHERE month(rental_date) = 5;
    """, engine)
rentals_may.head()

,rental_id,rental_date,inventory_id,customer_id,return_date,staff_id,last_update
0,1,2005-05-24 22:53:30,367,130,2005-05-26 22:04:30,1,2006-02-15 21:30:53
1,2,2005-05-24 22:54:33,1525,459,2005-05-28 19:40:33,1,2006-02-15 21:30:53
2,3,2005-05-24 23:03:39,1711,408,2005-06-01 22:12:39,1,2006-02-15 21:30:53
3,4,2005-05-24 23:04:41,2452,333,2005-06-03 01:43:41,2,2006-02-15 21:30:53
4,5,2005-05-24 23:05:21,2079,222,2005-06-02 04:33:21,1,2006-02-15 21:30:53


## Create a table `rentals_june` to store the data from rental table with information for the month of June.


In [5]:
rentals_june = ()

## Insert values in the table `rentals_june` using the table rental, filtering values only for the month of June.
 

In [6]:
# select all rentals from may that have either been rented or returned in june

rentals_june = pd.read_sql_query("""
    SELECT *
    FROM rental
    WHERE month(rental_date) = 6;
    """, engine)
rentals_june.head()

,rental_id,rental_date,inventory_id,customer_id,return_date,staff_id,last_update
0,1158,2005-06-14 22:53:33,1632,416,2005-06-18 21:37:33,2,2006-02-15 21:30:53
1,1159,2005-06-14 22:55:13,4395,516,2005-06-17 02:11:13,1,2006-02-15 21:30:53
2,1160,2005-06-14 23:00:34,2795,239,2005-06-18 01:58:34,2,2006-02-15 21:30:53
3,1161,2005-06-14 23:07:08,1690,285,2005-06-21 17:12:08,1,2006-02-15 21:30:53
4,1162,2005-06-14 23:09:38,987,310,2005-06-23 22:00:38,1,2006-02-15 21:30:53


## Check the number of rentals for each customer for May.


In [7]:
rentals_per_customer_may = pd.read_sql_query("""
    SELECT customer_id, count(rental_id)
    FROM rental
    WHERE month(rental_date) = 5
    GROUP BY customer_id;
    """, engine)
rentals_per_customer_may

,customer_id,count(rental_id)
0,1,2
1,2,1
2,3,2
3,5,3
4,6,3
...,...,...
515,594,4
516,595,1
517,596,6
518,597,2


## Check the number of rentals for each customer for June.
 

In [8]:
rentals_per_customer_june = pd.read_sql_query("""
    SELECT customer_id, count(rental_id)
    FROM rental
    WHERE month(rental_date) = 6
    GROUP BY customer_id;
    """, engine)
rentals_per_customer_june

,customer_id,count(rental_id)
0,1,7
1,2,1
2,3,4
3,4,6
4,5,5
...,...,...
585,595,2
586,596,2
587,597,3
588,598,1


## Create a Python connection with SQL database and retrieve the results of the last two queries (also mentioned below) as dataframes:

  - Check the number of rentals for each customer for May
  - Check the number of rentals for each customer for June

    **Hint**: You can store the results from the two queries in two separate dataframes.
 

In [9]:
# I did not understand the difference between this and the previous exercise

## Write a function that checks if customer borrowed more or less films in the month of June as compared to May.
 
 **Hint**: For this part, you can create a join between the two dataframes created before, using the merge function available for pandas dataframes. Here is a link to the documentation for the [merge function](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.merge.html).

In [10]:
# merge both dataframes
comparison = rentals_per_customer_may.merge(rentals_per_customer_june, on='customer_id')

comparison

,customer_id,count(rental_id)_x,count(rental_id)_y
0,1,2,7
1,2,1,1
2,3,2,4
3,5,3,5
4,6,3,4
...,...,...,...
507,594,4,6
508,595,1,2
509,596,6,2
510,597,2,3


In [11]:
# rename columns for better readability

comparison.columns = ["customer_id", "rentals_may", "rentals_june"]
comparison

,customer_id,rentals_may,rentals_june
0,1,2,7
1,2,1,1
2,3,2,4
3,5,3,5
4,6,3,4
...,...,...,...
507,594,4,6
508,595,1,2
509,596,6,2
510,597,2,3


In [12]:
# add new column for comparison
# positive values are higher than may, negative values are lower than may

comparison["high_low"] = comparison["rentals_june"] - comparison["rentals_may"] 

comparison.head()

,customer_id,rentals_may,rentals_june,high_low
0,1,2,7,5
1,2,1,1,0
2,3,2,4,2
3,5,3,5,2
4,6,3,4,1


In [15]:
# solution with function
# add temp_data to not mess with dataframe

def rental_check(x):
    if comparison["rentals_june"] > comparison["rentals_may"]:
        return True
    else: 
        return False

comparison['high_low_func'] = comparison['high_low_func'].apply(rental_check)
    
comparison.head()

KeyError: 'high_low_func'